# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import numpy as np
import torch
from datasets import load_dataset

device = "cpu"
#if (torch.cuda.is_available()): device="cuda"
print(f"device: {device}")
model = AutoModelForSequenceClassification.from_pretrained("gpt2") #AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
# Add a new padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Optionally, set the padding token to the newly added token
tokenizer.pad_token = tokenizer.eos_token 

dataset = load_dataset("agentlans/grammar-classification")
dataset["train"][0]

device: cpu


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

{'text': 'We have carried out a transaction cost analysis to help us understand the quality of execution on the execution venues where we execute/place orders.',
 'grammar': 1}

In [2]:
def tokenizing_map(dataset_entry):
    return tokenizer(dataset_entry["text"], padding=True, truncation=True, return_tensors="pt").to(device)

# temp small datasets to quickly iterate on config
small_validation_dataset = dataset["validation"].select(range(500)).rename_column("grammar","labels")
small_train_dataset = dataset["train"].select(range(500)).rename_column("grammar","labels")

small_validation_dataset=small_validation_dataset.map(tokenizing_map, batched=True)
small_train_dataset=small_train_dataset.map(tokenizing_map, batched=True)

for param in model.base_model.parameters():
    param.requires_grad = False
    
small_train_dataset

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 500
})

In [3]:
from datasets import load_metric

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
   
    return {
        "accuracy": (predictions == labels).mean(),
    }

trainer = Trainer(
    model=model,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    args=TrainingArguments(
        output_dir="./data/grammar",
        evaluation_strategy="epoch",
        num_train_epochs=3,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
    ),
    train_dataset=small_train_dataset,
    eval_dataset=small_validation_dataset,
    compute_metrics=compute_metrics
)
# trainer.train()
trainer.evaluate()

[[4821, 284, 35692, 11, 262, 3241, 286, 5386, 743, 307, 530, 286, 257, 749, 1593, 7612, 286, 852, 281, 4050, 10834, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], [464, 1570, 318, 4855, 416, 2351, 5136, 329, 3893, 4992, 357, 22125, 17184, 8, 37322, 341, 329, 26935, 287, 27684, 3893, 4992, 290, 7276, 2688, 357, 16827, 39, 7397, 2688, 8, 379, 2059, 39946, 

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 3.8696773052215576,
 'eval_accuracy': 0.478,
 'eval_runtime': 7.8726,
 'eval_samples_per_second': 63.512,
 'eval_steps_per_second': 63.512}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [4]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    task_type="CAUSAL_LM",
    target_modules=["c_attn","c_proj"]
)

lora_model = get_peft_model(model, config)
lora_model.to(device)
print("done")

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


done


In [7]:
from transformers import default_data_collator
lora_trainer = Trainer(
    model=lora_model,
    data_collator=default_data_collator,
    args=TrainingArguments(
        output_dir="./data/grammar",
        evaluation_strategy="epoch",
        num_train_epochs=5,
        learning_rate=2e-4,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
    ),
    train_dataset=small_train_dataset,
    eval_dataset=small_validation_dataset,
    compute_metrics=compute_metrics
)
lora_model.print_trainable_parameters()
# print ( lora_trainer.train_dataset[0])
# print((small_train_dataset[0]))
# print((small_validation_dataset[0]))
lora_trainer.train()
lora_trainer.evaluate()

trainable params: 811,008 || all params: 125,252,352 || trainable%: 0.6474992182182735


Epoch,Training Loss,Validation Loss,Accuracy
1,1.193600,0.805055,0.546000
2,0.965400,0.759555,0.570000
3,0.854200,0.759362,0.630000
4,0.917600,1.060125,0.656000
5,0.801400,1.453814,0.608000


Checkpoint destination directory ./data/grammar/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/grammar/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/grammar/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/grammar/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/grammar/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 1.4538136720657349,
 'eval_accuracy': 0.608,
 'eval_runtime': 11.2036,
 'eval_samples_per_second': 44.628,
 'eval_steps_per_second': 44.628,
 'epoch': 5.0}

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [6]:
# Saving the model
44model.save("/tmp/your_model_name")

SyntaxError: invalid decimal literal (684800285.py, line 2)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.